# Importing all the libraries

In [1]:
import pandas as pd
import numpy as np

# Getting the Dataset

In [2]:
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

In [3]:
data = pd.concat([test.assign(ind="test"), train.assign(ind="train")])

In [4]:
data.head(n=10)

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,ind,survived
0,1.0,"Flynn, Mr. John Irwin (""Irving"")",male,36.0,0.0,0.0,PC 17474,26.3875,E25,S,5,NaN,"Brooklyn, NY",test,NaN
1,3.0,"Sage, Miss. Constance Gladys",female,NaN,8.0,2.0,CA. 2343,69.5500,NaN,S,NaN,NaN,NaN,test,NaN
2,1.0,"Rood, Mr. Hugh Roscoe",male,NaN,0.0,0.0,113767,50.0000,A32,S,NaN,NaN,"Seattle, WA",test,NaN
3,2.0,"Gillespie, Mr. William Henry",male,34.0,0.0,0.0,12233,13.0000,NaN,S,NaN,NaN,"Vancouver, BC",test,NaN
4,2.0,"Collander, Mr. Erik Gustaf",male,28.0,0.0,0.0,248740,13.0000,NaN,S,NaN,NaN,"Helsinki, Finland Ashtabula, Ohio",test,NaN
5,2.0,"Collyer, Mrs. Harvey (Charlotte Annie Tate)",female,31.0,1.0,1.0,C.A. 31921,26.2500,NaN,S,14,NaN,"Bishopstoke, Hants / Fayette Valley, ID",test,NaN
6,1.0,"Uruchurtu, Don. Manuel E",male,40.0,0.0,0.0,PC 17601,27.7208,NaN,C,NaN,NaN,"Mexico City, Mexico",test,NaN
7,3.0,"Razi, Mr. Raihed",male,NaN,0.0,0.0,2629,7.2292,NaN,C,NaN,NaN,NaN,test,NaN
8,3.0,"Davies, Mr. Evan",male,22.0,0.0,0.0,SC/A4 23568,8.0500,NaN,S,NaN,NaN,NaN,test,NaN
9,1.0,"Loring, Mr. Joseph Holland",male,30.0,0.0,0.0,113801,45.5000,NaN,S,NaN,NaN,"London / New York, NY",test,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1008
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   float64
 1   name       1309 non-null   object 
 2   sex        1309 non-null   object 
 3   age        1046 non-null   float64
 4   sibsp      1309 non-null   float64
 5   parch      1309 non-null   float64
 6   ticket     1309 non-null   object 
 7   fare       1308 non-null   float64
 8   cabin      295 non-null    object 
 9   embarked   1307 non-null   object 
 10  boat       486 non-null    object 
 11  body       121 non-null    float64
 12  home.dest  745 non-null    object 
 13  ind        1309 non-null   object 
 14  survived   1009 non-null   float64
dtypes: float64(7), object(8)
memory usage: 163.6+ KB


In [6]:
columns_to_drop = ["name","cabin","home.dest","ticket","boat","body","embarked"]

In [7]:
data = data.drop(columns_to_drop,axis=1)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1008
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   float64
 1   sex       1309 non-null   object 
 2   age       1046 non-null   float64
 3   sibsp     1309 non-null   float64
 4   parch     1309 non-null   float64
 5   fare      1308 non-null   float64
 6   ind       1309 non-null   object 
 7   survived  1009 non-null   float64
dtypes: float64(6), object(2)
memory usage: 92.0+ KB


# Filling Nan Values

In [9]:
#print(len(data[data.embarked == "Q"]),len(data[data.embarked == "S"]),len(data[data.embarked == "C"]))
nan_columns = {"parch":data["parch"].mean(),"fare":data["fare"].mean(),"age":data["age"].mean()}

In [10]:
data = data.fillna(value = nan_columns)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1008
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   float64
 1   sex       1309 non-null   object 
 2   age       1309 non-null   float64
 3   sibsp     1309 non-null   float64
 4   parch     1309 non-null   float64
 5   fare      1309 non-null   float64
 6   ind       1309 non-null   object 
 7   survived  1009 non-null   float64
dtypes: float64(6), object(2)
memory usage: 92.0+ KB


In [12]:
print(data.shape)

(1309, 8)


# Label Encoding

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
le = LabelEncoder()

In [15]:
data["sex"] =le.fit_transform(data["sex"])

In [16]:
#data["embarked"] =le.fit_transform(data["embarked"])

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1008
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   float64
 1   sex       1309 non-null   int64  
 2   age       1309 non-null   float64
 3   sibsp     1309 non-null   float64
 4   parch     1309 non-null   float64
 5   fare      1309 non-null   float64
 6   ind       1309 non-null   object 
 7   survived  1009 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 92.0+ KB


In [18]:
data.head()
input_columns = ["pclass","sex","age","sibsp","parch","fare"]

In [19]:
test, train = data[data["ind"].eq("test")], data[data["ind"].eq("train")]

In [20]:
def entropy(col):
  counts = np.unique(col,return_counts = True)
  N = float(col.shape[0])
  ent = 0.0
  for ix in counts[1]:
    p = ix/N
    ent += (-1.0*p*np.log2(p))
  return ent

In [21]:
def divide_data(x_data,fkey,fval):
  x_right = pd.DataFrame([],columns=x_data.columns)
  x_left = pd.DataFrame([],columns = x_data.columns)
  for ix in range(x_data.shape[0]):
    val = x_data[fkey].loc[ix]
    if val>fval:
      x_right = x_right.append(x_data.loc[ix])
    else:
      x_left =  x_left.append(x_data.loc[ix])
  return x_left,x_right


In [22]:
def information_gain(x_data,fkey,fval):
  left,right = divide_data(x_data,fkey,fval)
  l = float(left.shape[0])/x_data.shape[0]
  r = float(right.shape[0])/x_data.shape[0]
  if left.shape[0] == 0 or right.shape[0] == 0:
    return -1000
  i_gain = entropy(x_data.survived)-(l*entropy(left.survived))-(r*entropy(right.survived))
  return i_gain

In [23]:

#train = train.drop(columns = ["ind"],axis=1)
print(train)
for fx in input_columns:
  print(information_gain(train,fx,train[fx].mean()))

      pclass  sex        age  sibsp  parch      fare    ind  survived
0        3.0    0  29.881135    0.0    0.0    7.7500  train       0.0
1        2.0    1  39.000000    0.0    0.0   26.0000  train       0.0
2        2.0    0  40.000000    0.0    0.0   13.0000  train       1.0
3        3.0    0  31.000000    1.0    1.0   20.5250  train       1.0
4        3.0    0  29.881135    2.0    0.0   23.2500  train       1.0
...      ...  ...        ...    ...    ...       ...    ...       ...
1004     1.0    1  40.000000    0.0    0.0   31.0000  train       1.0
1005     3.0    0  37.000000    0.0    0.0    9.5875  train       0.0
1006     1.0    0  23.000000    1.0    0.0  113.2750  train       1.0
1007     3.0    1  12.000000    1.0    0.0   11.2417  train       1.0
1008     3.0    1  30.000000    0.0    0.0    8.0500  train       0.0

[1009 rows x 8 columns]
0.055456910002982474
0.19274737190850932
0.0010525742338489685
0.006492394392888956
0.019756080122948244
0.042427934014281715


In [24]:
class Decisiontree:
  def __init__(self,depth=0,max_depth=10):
    self.left = None
    self.right = None
    self.fkey = None
    self.fval = None
    self.max_depth = max_depth
    self.depth = depth

  def trainer(self,X_train):
    features = input_columns
    info_gains = []
    print(features)
    for ix in features:
      i_gain = information_gain(X_train,ix,X_train[ix].mean())
      info_gains.append(i_gain)
    self.fkey = features[np.argmax(info_gains)]
    self.fval = X_train[self.fkey].mean()
     
    

    data_left,data_right = divide_data(X_train,self.fkey,self.fval)
    data_left = data_left.reset_index(drop=True)
    data_right = data_right.reset_index(drop=True)

    print(self.fkey,data_left.shape[0],data_right.shape[0])
    if data_left.shape[0] == 0 or data_right.shape[0] == 0:
      if X_train.survived.mean()>=0.5:
        self.target = 1.0
      else:
        self.target = 0.0
      return


    if (self.depth>=self.max_depth):
      if X_train.survived.mean()>=0.5:
        self.target = 1.0
      else:
        self.target = 0.0
      return
    
    self.left = Decisiontree(depth=self.depth+1,max_depth = self.max_depth)
    self.right = Decisiontree(depth = self.depth+1,max_depth = self.max_depth)
    self.left.trainer(data_left)
    self.right.trainer(data_right)

    if X_train.survived.mean()>=0.5:
      self.target = 1.0
    else:
      self.target = 0.0
    return
  
  def predict(self,test):
    if test[self.fkey]>self.fval:
      if self.right is None:
        return self.target
      return self.right.predict(test)
    else:
      if self.left is None:
        return self.target
      return self.left.predict(test)


In [25]:
dt = Decisiontree()
dt.trainer(train)


['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
sex 361 648
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
pclass 191 170
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
pclass 116 75
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
fare 77 39
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
sibsp 38 39
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
age 24 14
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
age 13 11
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
age 5 8
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
age 3 2
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
parch 1 2
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
pclass 1 0
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
fare 1 1
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
age 1 1
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
pclass 1 0
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
pclass 1 0
['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare']
age 4 4
['pclass', 'sex'

[[ 1.          1.         36.          0.          0.         26.3875    ]
 [ 3.          0.         29.88113451  8.          2.         69.55      ]
 [ 1.          1.         29.88113451  0.          0.         50.        ]
 ...
 [ 3.          1.         13.          4.          2.         31.3875    ]
 [ 3.          1.         29.88113451  2.          0.         21.6792    ]
 [ 3.          0.         29.88113451  0.          0.          7.75      ]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
sk_tree = RandomForestClassifier(n_estimators = 15,criterion = "entropy",max_depth=20)


In [ ]:
sk_tree.fit(train[input_columns],train["survived"])

In [ ]:
pred = list(sk_tree.predict(test[input_columns]))

In [ ]:
import csv
with open("output.csv","w") as f:
    csv_writer = csv.writer(f)
    header = ["Id","survived"]
    csv_writer.writerow(header)
    for i in range(len(pred)):
        csv_writer.writerow([str(i),str(pred[i])])

In [ ]:
v = pd.read_csv("output.csv")
print(v)